## NER Example on Current Structure

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/talukaragoz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df=pd.read_csv("tweet_fetch_050422.csv")
df['text']=df['text'].astype(str)
df['lowercase']=df['text'].map(lambda x: x.lower())

In [3]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",
    #    'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags. #IS IT GOOD IDEA TO KEEP? 
    unpack_contractions=False,  # Unpack contractions (can't -> can not). WORD EMBEDDINGS INCLUDE CONTRACTIONS
    spell_correct_elong=False,  # spell correction for elongated words. I'M THINKING IT MIGHT CAUSE TROUBLE WHEN SEVERAL LANGUAGES
                                #DONT THINK THERE WILL BE MANY ELONGATED WORDS IN ECONOMIC TWEETS
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    #dicts=[emoticons]   COMMENTED. LEAVES EMOTICONS AS EMOTICONS
)

/opt/anaconda3/envs/csfnlp/lib/python3.10/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/opt/anaconda3/envs/csfnlp/lib/python3.10/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [4]:
df['pota'] = df['lowercase'].map(text_processor.pre_process_doc)

In [5]:
en_tweets=df[df['lang']=='en']
de_tweets=df[df['lang']=='de']

en_stopwords=stopwords.words('english')
de_stopwords=stopwords.words('german')

en_tweets['tokenized_no_sw']=np.zeros(len(en_tweets["lowercase"]))
de_tweets['tokenized_no_sw']=np.zeros(len(de_tweets["lowercase"]))

en_tweets['tokenized_no_sw']=en_tweets['pota'].map(lambda x:[word for word in x if not word in en_stopwords])
de_tweets['tokenized_no_sw']=de_tweets['pota'].map(lambda x:[word for word in x if not word in de_stopwords])


/var/folders/y6/pght29y10p56pqbrywqnnfs40000gn/T/ipykernel_58613/3450864582.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_tweets['tokenized_no_sw']=np.zeros(len(en_tweets["lowercase"]))
/var/folders/y6/pght29y10p56pqbrywqnnfs40000gn/T/ipykernel_58613/3450864582.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  de_tweets['tokenized_no_sw']=np.zeros(len(de_tweets["lowercase"]))
/var/folders/y6/pght29y10p56pqbrywqnnfs40000gn/T/ipykernel_58613/3450864582.py:10: SettingWithCopyWarning: 
A value is try

In [6]:
en_tweets

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,lang,created_at,retweets,replies,likes,quote_count,lowercase,pota,tokenized_no_sw
0,0,1409940528284594180,serbiamontene11,9,1611,NaN,your house,Zimbabwe. Moscow had previously lifted curbs o...,en,2022-04-04 23:58:37+00:00,0.0,0.0,1.0,0.0,zimbabwe. moscow had previously lifted curbs o...,"[zimbabwe, ., moscow, had, previously, lifted,...","[zimbabwe, ., moscow, previously, lifted, curb..."
1,1,1426697206371848195,remin3d,49,544,i like to play games and talk movies. | never ...,he/it/xe,here's the link if anyone wants it! i've seen ...,en,2022-04-04 23:53:33+00:00,0.0,0.0,0.0,0.0,here's the link if anyone wants it! i've seen ...,"[here, ', s, the, link, if, anyone, wants, it,...","[', link, anyone, wants, !, ', seen, good, thi..."
2,2,859754699414798336,TheFrozenToes24,1554,5427,Xbox gt: FrozenToes24 |\nOwner of Shake N' Bak...,NaN,Round 2: SnB at Auto Club! https://t.co/yeedPu...,en,2022-04-04 23:39:45+00:00,0.0,1.0,5.0,0.0,round 2: snb at auto club! https://t.co/yeedpu...,"[round, <number>, :, snb, at, auto, club, !, <...","[round, <number>, :, snb, auto, club, !, <url>]"
3,3,1339389392,qayyumawan1010,94,22700,NaN,NaN,Turkish Switzerland AGP bus Car calculate tran...,en,2022-04-04 23:29:41+00:00,0.0,0.0,0.0,0.0,turkish switzerland agp bus car calculate tran...,"[turkish, switzerland, agp, bus, car, calculat...","[turkish, switzerland, agp, bus, car, calculat..."
5,5,910634761210785792,Temebrarius,1184,3298,🇩🇪 🇬🇧 German / British - Military Trauma Surge...,Ukraine,@walterlekh @nemapovnovazhen Her salary is ins...,en,2022-04-04 23:05:51+00:00,0.0,0.0,2.0,0.0,@walterlekh @nemapovnovazhen her salary is ins...,"[<user>, <user>, her, salary, is, insane, by, ...","[<user>, <user>, salary, insane, way, ,, tax, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,594,14770715,swissbusiness,87994,692881,The Swiss Business Club is connecting business...,Zurich,Calcium glycinate Market Outlook 2022 Analysis...,en,2022-04-04 00:10:02+00:00,0.0,0.0,0.0,0.0,calcium glycinate market outlook 2022 analysis...,"[calcium, glycinate, market, outlook, <number>...","[calcium, glycinate, market, outlook, <number>..."
599,595,3201524126,chinichijocson,21,2533,NaN,NaN,Chair Investment Division port user online tur...,en,2022-04-04 00:07:47+00:00,0.0,0.0,0.0,0.0,chair investment division port user online tur...,"[chair, investment, division, port, user, onli...","[chair, investment, division, port, user, onli..."
600,596,1211669817255575553,BRforJin,1416,51050,how to write blog 👉https://t.co/PF3Yux5Btn ❤️ ...,NaN,🗳\nUnlimited \n1⃣️ https://t.co/pbtRTZLyxk\n2⃣...,en,2022-04-04 00:07:16+00:00,3.0,0.0,6.0,0.0,🗳\nunlimited \n1⃣️ https://t.co/pbtrtzlyxk\n2⃣...,"[🗳, unlimited, <number>, ⃣, ️, <url>, <number>...","[🗳, unlimited, <number>, ⃣, ️, <url>, <number>..."
601,597,1416043380883263491,outlaw_barber,141,2682,Outlaw Barber Anarchist. Monero & Piratechain ...,NaN,@BIS_org @SNB_BNS Why are you a state like the...,en,2022-04-04 00:06:28+00:00,0.0,0.0,1.0,0.0,@bis_org @snb_bns why are you a state like the...,"[<user>, <user>, why, are, you, a, state, like...","[<user>, <user>, state, like, vatican, ?]"


In [8]:
from ner import remove_entities

In [9]:
import warnings
warnings.filterwarnings('ignore')

en_tweets["tokenized_no_sw_and_entities"] = np.zeros(len(en_tweets["lowercase"]))

for i in en_tweets.index:
    en_tweets["tokenized_no_sw_and_entities"][i] = remove_entities(en_tweets["text"][i], en_tweets["tokenized_no_sw"][i].copy())

In [14]:
en_tweets

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,lang,created_at,retweets,replies,likes,quote_count,lowercase,pota,tokenized_no_sw,tokenized_no_sw_and_entities
0,0,1409940528284594180,serbiamontene11,9,1611,NaN,your house,Zimbabwe. Moscow had previously lifted curbs o...,en,2022-04-04 23:58:37+00:00,0.0,0.0,1.0,0.0,zimbabwe. moscow had previously lifted curbs o...,"[zimbabwe, ., moscow, had, previously, lifted,...","[zimbabwe, ., moscow, previously, lifted, curb...","[., previously, lifted, curbs, air, links, <nu..."
1,1,1426697206371848195,remin3d,49,544,i like to play games and talk movies. | never ...,he/it/xe,here's the link if anyone wants it! i've seen ...,en,2022-04-04 23:53:33+00:00,0.0,0.0,0.0,0.0,here's the link if anyone wants it! i've seen ...,"[here, ', s, the, link, if, anyone, wants, it,...","[', link, anyone, wants, !, ', seen, good, thi...","[', link, anyone, wants, !, ', seen, good, thi..."
2,2,859754699414798336,TheFrozenToes24,1554,5427,Xbox gt: FrozenToes24 |\nOwner of Shake N' Bak...,NaN,Round 2: SnB at Auto Club! https://t.co/yeedPu...,en,2022-04-04 23:39:45+00:00,0.0,1.0,5.0,0.0,round 2: snb at auto club! https://t.co/yeedpu...,"[round, <number>, :, snb, at, auto, club, !, <...","[round, <number>, :, snb, auto, club, !, <url>]","[<number>, :, snb, !, <url>]"
3,3,1339389392,qayyumawan1010,94,22700,NaN,NaN,Turkish Switzerland AGP bus Car calculate tran...,en,2022-04-04 23:29:41+00:00,0.0,0.0,0.0,0.0,turkish switzerland agp bus car calculate tran...,"[turkish, switzerland, agp, bus, car, calculat...","[turkish, switzerland, agp, bus, car, calculat...","[bus, car, calculate, transmitting, market, gr..."
5,5,910634761210785792,Temebrarius,1184,3298,🇩🇪 🇬🇧 German / British - Military Trauma Surge...,Ukraine,@walterlekh @nemapovnovazhen Her salary is ins...,en,2022-04-04 23:05:51+00:00,0.0,0.0,2.0,0.0,@walterlekh @nemapovnovazhen her salary is ins...,"[<user>, <user>, her, salary, is, insane, by, ...","[<user>, <user>, salary, insane, way, ,, tax, ...","[<user>, <user>, salary, insane, way, ,, tax, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,594,14770715,swissbusiness,87994,692881,The Swiss Business Club is connecting business...,Zurich,Calcium glycinate Market Outlook 2022 Analysis...,en,2022-04-04 00:10:02+00:00,0.0,0.0,0.0,0.0,calcium glycinate market outlook 2022 analysis...,"[calcium, glycinate, market, outlook, <number>...","[calcium, glycinate, market, outlook, <number>...","[calcium, glycinate, market, outlook, <number>..."
599,595,3201524126,chinichijocson,21,2533,NaN,NaN,Chair Investment Division port user online tur...,en,2022-04-04 00:07:47+00:00,0.0,0.0,0.0,0.0,chair investment division port user online tur...,"[chair, investment, division, port, user, onli...","[chair, investment, division, port, user, onli...","[port, user, online, turquoise, data, port, <u..."
600,596,1211669817255575553,BRforJin,1416,51050,how to write blog 👉https://t.co/PF3Yux5Btn ❤️ ...,NaN,🗳\nUnlimited \n1⃣️ https://t.co/pbtRTZLyxk\n2⃣...,en,2022-04-04 00:07:16+00:00,3.0,0.0,6.0,0.0,🗳\nunlimited \n1⃣️ https://t.co/pbtrtzlyxk\n2⃣...,"[🗳, unlimited, <number>, ⃣, ️, <url>, <number>...","[🗳, unlimited, <number>, ⃣, ️, <url>, <number>...","[🗳, unlimited, <number>, ⃣, ️, <url>, <number>..."
601,597,1416043380883263491,outlaw_barber,141,2682,Outlaw Barber Anarchist. Monero & Piratechain ...,NaN,@BIS_org @SNB_BNS Why are you a state like the...,en,2022-04-04 00:06:28+00:00,0.0,0.0,1.0,0.0,@bis_org @snb_bns why are you a state like the...,"[<user>, <user>, why, are, you, a, state, like...","[<user>, <user>, state, like, vatican, ?]","[<user>, <user>, state, like, ?]"


In [13]:
de_tweets["tokenized_no_sw_and_entities"] = np.zeros(len(de_tweets["lowercase"]))

for i in de_tweets.index:
    de_tweets["tokenized_no_sw_and_entities"][i] = remove_entities(de_tweets["text"][i], de_tweets["tokenized_no_sw"][i].copy())

In [15]:
de_tweets

,Unnamed: 0,author_id,username,author_followers,author_tweets,author_description,author_location,text,lang,created_at,retweets,replies,likes,quote_count,lowercase,pota,tokenized_no_sw,tokenized_no_sw_and_entities
4,4,864409134594220032,mgarrtner,731,8760,Information Archaeologist - Treasure Hunter - ...,Vienna - Berlin - Vladivostok,@Weisse_Kerze @sven34262416 @h0tzenpl0tzz @And...,de,2022-04-04 23:20:01+00:00,0.0,1.0,1.0,0.0,@weisse_kerze @sven34262416 @h0tzenpl0tzz @and...,"[<user>, <user>, <user>, <user>, <user>, <user...","[<user>, <user>, <user>, <user>, <user>, <user...","[<user>, <user>, <user>, <user>, <user>, <user..."
10,10,756520461547212807,rubu481,444,34683,Only doing what is #TheRightThingToDo #StopASS...,NaN,Mit dem Export von Gas und Öl finanziert Russl...,de,2022-04-04 22:43:36+00:00,0.0,0.0,0.0,0.0,mit dem export von gas und öl finanziert russl...,"[mit, dem, export, von, gas, und, öl, finanzie...","[export, gas, öl, finanziert, russland, krieg,...","[export, gas, öl, finanziert, russland, ., tag..."
11,11,1392204259610054657,Somewherefire,231,1263,NaN,Switzerland,@Blickch Nein wirklich? Vor 10 Stunden klang e...,de,2022-04-04 22:42:37+00:00,0.0,0.0,3.0,0.0,@blickch nein wirklich? vor 10 stunden klang e...,"[<user>, nein, wirklich, ?, vor, <number>, stu...","[<user>, nein, wirklich, ?, <number>, stunden,...","[<user>, wirklich, ?, <number>, beim, unabbhän..."
12,12,22781017,Blickch,270812,114988,"News & Recherchen von #Blick, #BlickTV & #Sonn...",Switzerland,Wenn der Westen ein Embargo gegen Putin beschl...,de,2022-04-04 22:34:51+00:00,0.0,14.0,4.0,0.0,wenn der westen ein embargo gegen putin beschl...,"[wenn, der, westen, ein, embargo, gegen, putin...","[westen, embargo, putin, beschliesst, :, schwe...","[beschliesst, :, droht, gas, -, fiasko, <url>]"
17,17,3946109487,RTO_Kollegin,1970,15091,"Aktuelle #Bordell & #ErotikJobs, #Vermietungen...",Europe,"Viele Gäste, viel Geld!! Bei Pameland®️ verdie...",de,2022-04-04 22:20:21+00:00,0.0,0.0,0.0,0.0,"viele gäste, viel geld!! bei pameland®️ verdie...","[viele, gäste, ,, viel, geld, !, !, bei, pamel...","[viele, gäste, ,, geld, !, !, pameland, ®️, ve...","[,, geld, !, !, pameland, ®️, verdient, wirkli..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,541,1411597949751238656,Housekeeper177,42,1116,NaN,Schweiz,@bruno_buchs @EDA_DFAE @ignaziocassis Auf die ...,de,2022-04-04 04:41:04+00:00,0.0,2.0,2.0,0.0,@bruno_buchs @eda_dfae @ignaziocassis auf die ...,"[<user>, <user>, <user>, auf, die, tradition, ...","[<user>, <user>, <user>, tradition, wegschauen...","[<user>, <user>, <user>, tradition, wegschauen..."
567,563,1423725726918975491,blub777777777,3,71,NaN,NaN,@erikschwarzdorn @ThorstenPolleit Da lohnt sic...,de,2022-04-04 02:37:40+00:00,0.0,0.0,1.0,0.0,@erikschwarzdorn @thorstenpolleit da lohnt sic...,"[<user>, <user>, da, lohnt, sich, meines, erac...","[<user>, <user>, lohnt, erachtens, blick, länd...","[<user>, <user>, lohnt, erachtens, blick, ., b..."
585,581,1108705216625553411,PamelaWarnstadt,2466,12502,Pamela Warnstadt Clever Passive verdienen - ge...,Spain,HEIMARBEIT-ONLINE-HEIMARBEIT\n\nhttps://t.co/9...,de,2022-04-04 01:15:00+00:00,0.0,0.0,0.0,0.0,heimarbeit-online-heimarbeit\n\nhttps://t.co/9...,"[heimarbeit, -, online, -, heimarbeit, <url>, ...","[heimarbeit, -, online, -, heimarbeit, <url>, ...","[heimarbeit, -, online, -, heimarbeit, <url>, ..."
587,583,272502906,GainMoneyFaster,5427,35576,𝙈𝙤𝙣𝙚𝙮 𝘾𝙤𝙖𝙘𝙝 showing you best Clever Passive In...,Bilbao,HEIMARBEIT-ONLINE-HEIMARBEIT\n\nhttps://t.co/S...,de,2022-04-04 01:00:01+00:00,0.0,0.0,0.0,0.0,heimarbeit-online-heimarbeit\n\nhttps://t.co/s...,"[heimarbeit, -, online, -, heimarbeit, <url>, ...","[heimarbeit, -, online, -, heimarbeit, <url>, ...","[heimarbeit, -, online, -, heimarbeit, <url>, ..."
